#Module 2: Algorithmic Thinking

In [115]:
from __future__ import division
import urllib2
import random
import time
import math
import matplotlib.pyplot as plt
from collections import Counter
import mod_2_hmwk
random.seed(1)

In [7]:
def copy_graph(graph):
    """
    Make a copy of a graph
    """
    new_graph = {}
    for node in graph:
        new_graph[node] = set(graph[node])
    return new_graph

def delete_node(ugraph, node):
    """
    Delete a node from an undirected graph
    """
    neighbors = ugraph[node]
    ugraph.pop(node)
    for neighbor in neighbors:
        ugraph[neighbor].remove(node)
    
def targeted_order(ugraph):
    """
    Compute a targeted attack order consisting
    of nodes of maximal degree
    
    Returns:
    A list of nodes
    """
    # copy the graph
    new_graph = copy_graph(ugraph)
    
    order = []    
    while len(new_graph) > 0:
        max_degree = -1
        for node in new_graph:
            if len(new_graph[node]) > max_degree:
                max_degree = len(new_graph[node])
                max_degree_node = node
        
        neighbors = new_graph[max_degree_node]
        new_graph.pop(max_degree_node)
        for neighbor in neighbors:
            new_graph[neighbor].remove(max_degree_node)

        order.append(max_degree_node)
    return order
    


##########################################################
# Code for loading computer network graph

NETWORK_URL = "http://storage.googleapis.com/codeskulptor-alg/alg_rf7.txt"


def load_graph(graph_url):
    """
    Function that loads a graph given the URL
    for a text representation of the graph
    
    Returns a dictionary that models a graph
    """
    graph_file = urllib2.urlopen(graph_url)
    graph_text = graph_file.read()
    graph_lines = graph_text.split('\n')
    graph_lines = graph_lines[ : -1]
    
    print "Loaded graph with", len(graph_lines), "nodes"
    
    answer_graph = {}
    for line in graph_lines:
        neighbors = line.split(' ')
        node = int(neighbors[0])
        answer_graph[node] = set([])
        for neighbor in neighbors[1 : -1]:
            answer_graph[node].add(int(neighbor))

    return answer_graph

In [60]:
def ER_graph(num_nodes):
    p = .0011
    i_j_pairs = [(i,j) for i in range(1,num_nodes+1) for j in range(1,num_nodes+1) if i!=j]
    ER_graph = {}
    for pair in i_j_pairs:
        if random.random()<p:
            if pair[0] not in ER_graph.keys():
                ER_graph[pair[0]] = set([])
                ER_graph[pair[0]].add(pair[1])
            else:
                ER_graph[pair[0]].add(pair[1])
            if pair[1] not in ER_graph.keys():
                ER_graph[pair[1]] = set([])
                ER_graph[pair[1]].add(pair[0])
            else:
                ER_graph[pair[0]].add(pair[0])
    return ER_graph

In [96]:
class UPATrial:
    """
    Simple class to encapsulate optimizated trials for the UPA algorithm
    
    Maintains a list of node numbers with multiple instance of each number.
    The number of instances of each node number are
    in the same proportion as the desired probabilities
    
    Uses random.choice() to select a node number from this list for each trial.
    """

    def __init__(self, num_nodes):
        """
        Initialize a UPATrial object corresponding to a 
        complete graph with num_nodes nodes
        
        Note the initial list of node numbers has num_nodes copies of
        each node number
        """
        self._num_nodes = num_nodes
        self._node_numbers = [node for node in range(num_nodes) for dummy_idx in range(num_nodes)]


    def run_trial(self, num_nodes):
        """
        Conduct num_nodes trials using by applying random.choice()
        to the list of node numbers
        
        Updates the list of node numbers so that each node number
        appears in correct ratio
        
        Returns:
        Set of nodes
        """
        
        # compute the neighbors for the newly-created node
        new_node_neighbors = set()
        for _ in range(num_nodes):
            new_node_neighbors.add(random.choice(self._node_numbers))
        
        # update the list of node numbers so that each node number 
        # appears in the correct ratio
        self._node_numbers.append(self._num_nodes)
        for dummy_idx in range(len(new_node_neighbors)):
            self._node_numbers.append(self._num_nodes)
        self._node_numbers.extend(list(new_node_neighbors))
        
        #update the number of nodes
        self._num_nodes += 1
        return new_node_neighbors
    
    def complete_trial(self,m,n=0):
        """
        adds nodes to graph as expected
        """
        if n==0:
            n = self._num_nodes
        for index in range(m,n):
            self.run_trial(index)

In [97]:
graph = load_graph(NETWORK_URL)

Loaded graph with 1239 nodes


In [98]:
num_nodes = 1239

In [99]:
ER_G = ER_graph(num_nodes)

In [100]:
sum(map(len,ER_G.values()))

3036

In [111]:
UPA_G = UPATrial(num_nodes)

In [107]:
m = int(math.floor(num_nodes*.90))

In [108]:
UPA_G.complete_trial(m)

1115 2169


In [112]:
counter_obj = Counter(UPA_G._node_numbers)

In [113]:
sum(counter_obj.values())

1535121

In [116]:
def random_order(graph):
    """
    returns a list of the nodes in random order from graph
    """
    nodes = graph.keys()
    random.shuffle(nodes)
    return nodes

In [ ]:
mod_2_hmwk.compute_resilience(ER_G,random_order(ER_G))